In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1265, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1265 (delta 85), reused 0 (delta 0), pack-reused 1151
Receiving objects: 100% (1265/1265), 15.45 MiB | 23.58 MiB/s, done.
Resolving deltas: 100% (915/915), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 23.2 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_puresvd.db',
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_puresvd.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [14]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective_function_pure_svd_item(trial):
    params = {
        'topK': trial.suggest_int('topK', 1, 1000, log=True),
        'num_factors': trial.suggest_int('num_factors', 1, 1000, log=True)
    }
                          
    recommender_instance = PureSVDItemRecommender(URM_train)
    recommender_instance.fit(**params)       
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    return result_df.loc[10, "MAP"]

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [15]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_puresvd_item',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_pure_svd_item, n_trials=200)

[I 2023-12-01 14:04:27,661] Using an existing study with name 'hyperparameters_tuning_puresvd_item' instead of creating a new one.


PureSVDItemRecommender: URM Detected 627 ( 4.8%) users with no interactions.
PureSVDItemRecommender: URM Detected 247 ( 1.1%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 10456 (100.0%) in 14.17 sec. Users per second: 738


[I 2023-12-01 14:05:10,755] Trial 300 finished with value: 0.03331249013249808 and parameters: {'topK': 56, 'num_factors': 183}. Best is trial 293 with value: 0.0337220939446938.


PureSVDItemRecommender: URM Detected 627 ( 4.8%) users with no interactions.
PureSVDItemRecommender: URM Detected 247 ( 1.1%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 10456 (100.0%) in 12.39 sec. Users per second: 844


[I 2023-12-01 14:05:52,216] Trial 301 finished with value: 0.032951563467774524 and parameters: {'topK': 30, 'num_factors': 250}. Best is trial 293 with value: 0.0337220939446938.


...

[I 2023-12-01 16:18:22,267] Trial 497 finished with value: 0.032762866463851806 and parameters: {'topK': 89, 'num_factors': 346}. Best is trial 293 with value: 0.0337220939446938.


PureSVDItemRecommender: URM Detected 627 ( 4.8%) users with no interactions.
PureSVDItemRecommender: URM Detected 247 ( 1.1%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 10456 (100.0%) in 13.46 sec. Users per second: 777


[I 2023-12-01 16:18:55,892] Trial 498 finished with value: 0.03172739355242251 and parameters: {'topK': 43, 'num_factors': 75}. Best is trial 293 with value: 0.0337220939446938.


PureSVDItemRecommender: URM Detected 627 ( 4.8%) users with no interactions.
PureSVDItemRecommender: URM Detected 247 ( 1.1%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 10456 (100.0%) in 11.44 sec. Users per second: 914


[I 2023-12-01 16:19:28,963] Trial 499 finished with value: 0.03346396767709925 and parameters: {'topK': 19, 'num_factors': 164}. Best is trial 293 with value: 0.0337220939446938.


In [16]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [17]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [18]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [19]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_puresvd_item.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_puresvd_item.json', 
            'tuning_results/best_params_puresvd_item.json', 
            'User KNN tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 
        '/kaggle/working/best_params_puresvd_item.json'
    )

In [20]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_puresvd.db',
        'Tuning db updated results (from kaggle notebook)'
    )